In [25]:
import pandas as pd

raw_path = "../data/raw/Fluo-C2DL-MSC/01/MSC01_allspots.csv"
df = pd.read_csv(raw_path)

df.shape, df.columns

print(df.head)

<bound method NDFrame.head of       LABEL       ID  TRACK_ID             QUALITY          POSITION_X  \
0     Label  Spot ID  Track ID             Quality                   X   
1     Label  Spot ID  Track ID             Quality                   X   
2       NaN      NaN       NaN           (quality)                (µm)   
3    ID8768     8768         1  3.1028220653533936   255.7899124533218   
4    ID8760     8760       NaN   2.018190860748291  163.80000639843774   
..      ...      ...       ...                 ...                 ...   
497  ID9158     9158        33  4.9127373695373535   75.44518282491416   
498  ID9159     9159        32   2.278413772583008  211.96917203302235   
499  ID9160     9160        10   3.291257381439209    74.4939169150496   
500  ID9161     9161        27  3.2306931018829346  138.84652866861424   
501  ID9162     9162        29   2.838289737701416  29.908978658518336   

             POSITION_Y POSITION_Z POSITION_T  FRAME  RADIUS  VISIBILITY  \
0    

In [24]:
spots = df.iloc[3:].copy()

print(spots.shape)
spots.head()

(499, 20)


,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,VISIBILITY,MANUAL_SPOT_COLOR,MEAN_INTENSITY_CH1,MEDIAN_INTENSITY_CH1,MIN_INTENSITY_CH1,MAX_INTENSITY_CH1,TOTAL_INTENSITY_CH1,STD_INTENSITY_CH1,CONTRAST_CH1,SNR_CH1
3,ID8768,8768,1,3.1028220653533936,255.7899124533218,232.8780587992927,0.0,0.0,0,14.0,1,NaN,9186.939561217268,8704.0,4608.0,19712.0,6.4905728E7,1982.0431434597406,0.16485546280018842,1.3119553457960516
4,ID8760,8760,NaN,2.018190860748291,163.80000639843774,0.0,0.0,0.0,0,14.0,1,NaN,4766.920452937014,6656.0,0.0,14080.0,3.3678293E7,4286.3838290267095,0.07810308426740761,0.1611330911126302
5,ID8761,8761,3,3.2842626571655273,125.81495569676237,10.679019360229411,0.0,0.0,0,14.0,1,NaN,8723.43991507431,8192.0,0.0,29440.0,6.1631103E7,3255.7515663381723,0.23551761095030538,1.0215060865811512
6,ID8762,8762,4,7.225203037261963,35.26357360765852,31.171818306708087,0.0,0.0,0,14.0,1,NaN,10176.33517338995,8704.0,4352.0,29184.0,7.1895808E7,4096.50285745562,0.15940008729879554,0.6830670912110204
7,ID8763,8763,5,12.672812461853027,258.81498952582467,48.82335433865504,0.0,0.0,0,14.0,1,NaN,11847.96263269639,8960.0,5120.0,54016.0,8.3705856E7,7781.419719741127,0.22003055913471756,0.5491956942323825


In [29]:
num_cols = [
    "ID", "TRACK_ID", "QUALITY",
    "POSITION_X", "POSITION_Y", "POSITION_Z",
    "POSITION_T", "FRAME", "RADIUS", "VISIBILITY"
]

for c in num_cols:
    spots[c] = pd.to_numeric(spots[c], errors="coerce")

spots[num_cols].dtypes

ID              int64
TRACK_ID      float64
QUALITY       float64
POSITION_X    float64
POSITION_Y    float64
POSITION_Z    float64
POSITION_T    float64
FRAME           int64
RADIUS        float64
VISIBILITY      int64
dtype: object

In [30]:
spots["TRACK_ID"].isna().sum()

np.int64(25)

In [31]:
spots = spots[spots["TRACK_ID"].notna()].copy()
spots.shape

(474, 20)

In [37]:
cols_map = {
    "TRACK_ID":   "cell_id",
    "FRAME":      "frame",
    "POSITION_T": "t_min",
    "POSITION_X": "x_um",
    "POSITION_Y": "y_um",
}

tracks = spots[list(cols_map.keys())].rename(columns=cols_map)

print(tracks.head())
tracks.dtypes

   cell_id  frame  t_min        x_um        y_um
3      1.0      0    0.0  255.789912  232.878059
5      3.0      0    0.0  125.814956   10.679019
6      4.0      0    0.0   35.263574   31.171818
7      5.0      0    0.0  258.814990   48.823354
8      6.0      0    0.0   23.511274  139.972312


cell_id    float64
frame        int64
t_min      float64
x_um       float64
y_um       float64
dtype: object

In [42]:
tracks = tracks.sort_values(["cell_id", "frame"]).reset_index(drop=True)

tracks["cell_id"] = tracks["cell_id"].astype(int)
tracks["frame"] = tracks["frame"].astype(int)


print(tracks.dtypes)
tracks.head()

cell_id      int64
frame        int64
t_min      float64
x_um       float64
y_um       float64
dtype: object


,cell_id,frame,t_min,x_um,y_um
0,0,10,200.0,39.867660,224.393768
1,0,11,220.0,34.776408,227.566565
2,0,12,240.0,31.056212,229.721570
3,0,13,260.0,32.449259,230.256018
4,0,14,280.0,33.368621,230.260754


In [44]:
lengths = tracks.groupby("cell_id")["frame"].nunique()
lengths.describe()

count    35.000000
mean     13.542857
std      12.514328
min       2.000000
25%       4.500000
50%      10.000000
75%      18.500000
max      48.000000
Name: frame, dtype: float64

In [48]:
min_len = 10
good_ids = lengths[lengths >= min_len].index

tracks_filtered = tracks[tracks["cell_id"].isin(good_ids)].copy()
tracks_filtered = tracks_filtered.sort_values(["cell_id", "frame"]).reset_index(drop=True)

tracks_filtered["cell_id"].nunique(), tracks["cell_id"].nunique()


(18, 35)

In [50]:
out_path1 = "../data/MSC01_tracks_clean.csv"
out_path2 = "../data/MSC01_tracks_clean_min10.csv"

#tracks.to_csv(out_path1, index=False)
#tracks_filtered.to_csv(out_path2, index=False)